<h1>A portfolio pricing program</h1>

In this assignment, you will write a <span style="color:blue">portfolio pricing program</span>. The portfolio sits in the database <span style="color:blue">portfolio</span>. The database contains two tables: <span style="color:blue">stocks</span> and <span style="color:blue">holdings</span>. 
<p>
Currently, the data sits in a txt file <span style="color:blue">portfolio.txt</span> The code in the next two cells reads this file, creates the database and the tables, and populates them with the data. Experiment with querying the database to get a sense for the (simple!) structure of the database
<p>
You need to do the following:
<li>Run the code that creates the database (make sure that portfolio.txt is in the same directory as this program
<li>Write a function that calculates the net pnl (gain) on each stock in the portfolio
<p>
Python is a particularly good environment for working with multiple data sources and the purpose of this assignment is to get some practice combining data doing just that. I've done most of the work for you (setting up the database) but take a look at that code as well and play around with it a bit.

<h1>Create the database and set up a db and cursor object</h1>

In [1]:
import pymysql
name = 'portfolio'
db = pymysql.connect("localhost","root","mrjybHAHAHA" )
cursor = db.cursor()
query = 'DROP DATABASE IF EXISTS ' + name + ';'
cursor.execute(query)
query = "CREATE DATABASE IF NOT EXISTS " + name + ';'
cursor.execute(query)
query = "USE " + name + ';'
cursor.execute(query)



0

<h2>Read portfolio.txt and create the two tables stocks and holdings</h2>

In [2]:

file = "portfolio.txt"
with open(file,'r') as f:
    line_count = 0
    stocks_set = set()
    for line in f:
        line = line.strip()

        if line_count == 0:
            headers = line.split(':')
            headers = [x.replace(' ','_') for x in headers]
            query1 = "DROP TABLE IF EXISTS stocks;"
            query2 = "DROP TABLE IF EXISTS holdings"
            cursor.execute(query1)
            cursor.execute(query2)
            query1 = "CREATE TABLE IF NOT EXISTS stocks ("
            query1 += headers[0] + " VARCHAR(10),"
            query1 += headers[1] + " VARCHAR(30));"
            query2 = "CREATE TABLE IF NOT EXISTS holdings ("
            query2 += headers[0] + " VARCHAR(10),"
            query2 += headers[2] + " DECIMAL(10,2),"
            query2 += headers[3] + " INT,"
            query2 += headers[4] + " DATE);"
            cursor.execute(query1)
            cursor.execute(query2)
            line_count += 1
            continue        
        data = line.split(':')
        stock_info = (data[0],data[1])
        stocks_set.add(stock_info)
        holdings_query = 'INSERT INTO holdings VALUES ("'
        holdings_query +=data[0] + '",'
        holdings_query +=data[2] + ','
        holdings_query +=data[3] + ',"'
        holdings_query +=data[4] + '");'
        cursor.execute(holdings_query)
for s_info in stocks_set:
    stock_query = 'INSERT INTO stocks VALUES ("'
    stock_query += s_info[0] + '","'
    stock_query += s_info[1] +'");'
    cursor.execute(stock_query)
db.commit()
db.close()

/anaconda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1051, "Unknown table 'portfolio.stocks'")
  result = self._query(query)
/anaconda/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1051, "Unknown table 'portfolio.holdings'")
  result = self._query(query)


<h2>Write a function that calculates the dollar gain for each stock</h2>
Your function should return a dictionary with the company name as the key and the gain as the value. For example, for the given data set, and stock prices at the close of 9/27/2017, the function should return (your actual gains will be different because stock prices do change!):
<p>
{<br>
'Alphabet, Inc.': -3105.0,<br>
 'Apple, Inc.': 60110.0,<br>
 'General Electric': 12095.0,<br>
 'Goldman Sachs': 112416.5<br>
}
<p>
Notes:
<li>Write a function get_price(ticker) that gets the current stock price from the google finance page
<li>You'll need to use a JOIN to combine the two tables in the database
<li>cursor.execute returns DECIMAL and INT types. Convert these into float and int
<li>gain = price_now * share - purchase_price * shares. You need to accumulate these values for each stock

 
 

In [3]:
def get_pnl():
    import pymysql
    db = pymysql.connect("localhost","root","mrjybHAHAHA" )
    gain_dict = dict()
    cursor = db.cursor()
    query = """
    use portfolio
    """
    cursor.execute(query)
    query = """
    select * FROM holdings
    INNER JOIN stocks ON holdings.Ticker = stocks.Ticker
    """
    cursor.execute(query)
    table1 = cursor.fetchall()
    
    table_calc = []
    for x in table1:
        result = round((float(get_price(x[0])) - float(x[1])) * float(x[2]),1)
        table_calc.append((x[5], result))
        
    for z in table_calc:
        if gain_dict.get(z[0]) == None:
            gain_dict[z[0]] = z[1]
        else:
            gain_dict[z[0]] += z[1]
            
    return gain_dict

In [5]:
get_pnl()

{'Alphabet, Inc.': 20650.0,
 'Apple, Inc.': 62935.0,
 'General Electric': 11590.0,
 'Goldman Sachs': 125804.0}

<h2>Write a function that - given a ticker - get's the current price from google finance</h2>
Explore the html code at url below to figure out how to get the current price of the stock

In [4]:
def get_price(ticker):
    import requests
    from bs4 import BeautifulSoup
    url = 'https://finance.google.com/finance?q='+ticker
    content = requests.get(url).content.decode()
    result = BeautifulSoup(content,'lxml')
    price = result.find('span', class_='pr').get_text().strip('\n')
    #price = float(price_s)
    #return price 
    return price

#Test
get_price('AAPL')

'159.88'

In [ ]:
print(get_pnl())
""" RETURNS (format, values will differ):
{'Alphabet, Inc.': -3105.0,
 'Apple, Inc.': 60110.0,
 'General Electric': 12095.0,
 'Goldman Sachs': 112416.5}

"""